In [1]:
import pandas as pd

In [33]:
path = 'archive/'
data = pd.read_csv(path+'movies_metadata.csv')

C:\Users\KTDS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [38]:
data = data[['id','genres','vote_average','vote_count','popularity','title','overview']]

In [39]:
#불공정 데이터 처리
m = data['vote_count'].quantile(0.9)

In [27]:
data

,id,genres,vote_average,vote_count,popularity,title,overview
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.9469,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.0155,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",6.5,92.0,11.7129,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",6.1,34.0,3.85949,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",5.7,173.0,8.38752,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...
45461,439050,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",4.0,1.0,0.072051,Subdue,Rising and falling between a man and woman.
45462,111109,"[{'id': 18, 'name': 'Drama'}]",9.0,3.0,0.178241,Century of Birthing,An artist struggles to finish his work while a...
45463,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",3.8,6.0,0.903007,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,227506,[],0.0,0.0,0.003503,Satan Triumphant,"In a small town live two brothers, one a minis..."


In [40]:
data = data.loc[data['vote_count']>=m]

In [43]:
C = data['vote_average'].mean()

In [37]:
data['vote_average'].describe()

count    45460.000000
mean         5.618207
std          1.924216
min          0.000000
25%          5.000000
50%          6.000000
75%          6.800000
max         10.000000
Name: vote_average, dtype: float64

In [44]:
def weighted_rating(x, m = m, C = C):
    v = x['vote_count']
    R = x['vote_average']
    return (v /(v+m)*R) +(m /(m+v)*C)

In [47]:
data['score'] = data.apply(
    lambda x : weighted_rating(x),
    axis = 1
)

In [61]:
from ast import literal_eval


def safe_literal_eval(node):
    try:
        return literal_eval(node)
    except ValueError:
        print(node)
        return node 

data['genres'] = data['genres'].apply(
    lambda x : safe_literal_eval(x)
)

In [68]:
data['genres'] = data['genres'].apply(
    lambda x: [d['name'] for d in x]
)

In [71]:
data['genres'] = data['genres'].apply(
    lambda x: " ".join(x)
)

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count_vector = CountVectorizer(ngram_range=(1,3))

In [77]:
c_vector_genres = count_vector.fit_transform(data['genres'])

In [79]:
c_vector_genres.shape

(4555, 1053)

In [82]:
gerne_c_sim = cosine_similarity(c_vector_genres,c_vector_genres).argsort()[:,::-1]

In [101]:
data = data.reset_index(drop =False)

In [118]:
def get_recommend_moive_list(df, movie_title, top = 30):

    target_moive_index = df[df['title']== movie_title] .index.values

    sim_index = gerne_c_sim[target_moive_index, : top].reshape(-1)

    sim_index = sim_index[sim_index != target_moive_index]

    result = df.iloc[sim_index].sort_values('score',ascending = False)[:10]
  
    return result

In [119]:
recommand_movie = get_recommend_moive_list(data,movie_title = 'The Dark Knight Rises')

In [ ]:
data